# Test the Stream Index

## Check if index is up!

In [1]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "0PsTttc",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "2YIbFkvuRqaPWhEjuEYnwA",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['foodmap.isti.cnr.it'], http_auth=('elastic', 'changeme'), port=9200)
print es

<Elasticsearch([{u'host': u'foodmap.isti.cnr.it'}])>


## Index operations

#### Delete index, delete tweets by id and query

In [4]:
# es.indices.delete(index='stream', ignore=[400, 404])

{u'acknowledged': True}

In [ ]:
# es.delete_by_query(index='stream', doc_type='tweet', body={
#     'query': {
#         "match_all" : {}
#     }
# })

#### Count elements in index

In [35]:
es.count(index='stream', doc_type='tweet')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 2851}

#### GET all docs from index

In [8]:
res = es.search(index="stream", doc_type='tweet', size=100, body={"query": {"match_all": {}}})
# [x['_source'] for x in res['hits']['hits']]
for r in res['hits']['hits']:
    print r
    print 

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'880451272565964800', u'_source': {u'username': u'JuiceBarPlano', u'lang': u'en', u'text_categories': [u'spring rolls'], u'city': u'plano', u'text': u'Do you have plans for lunch?  We do! \U0001f609 #sproutsandwich #springrolls #minttobe  #juicebarplano\u2026 https://t.co/WCBVxLSGKc', u'img_categories': None, u'hashtags': [u'#sproutsandwich', u'#springrolls', u'#minttobe', u'#juicebarplano\u2026'], u'img_flag': True, u'timestamp_ms': u'1498750916030', u'bounding_box': {u'type': u'Polygon', u'coordinates': [[[-96.85743, 32.997151], [-96.85743, 33.125334], [-96.612371, 33.125334], [-96.612371, 32.997151]]]}, u'coords': [-96.7945949, 33.0508655], u'id_str': u'880451272565964800', u'country': u'united states', u'id': 880451272565964800, u'created_at_day': 20170629, u'created_at_datetime': u'2017-06-29T15:41:56', u'media_url': None}, u'_index': u'stream'}

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'880452525392097280', u'_source': {u'usernam

#### GET specific tweets

In [ ]:
es.get(index='stream', doc_type='tweet', id=880404048834416644)

In [ ]:
results = es.search(index='stream', doc_type='tweet', size=100, body={
  'query': {
    'match': {
      'text_categories': 'donuts',
#       'day': 20170618,
#         'sort': [
#         {'committed_date': {'order': 'desc'}}
#       ],
     }
  }
})

print results
print len(results["hits"]["hits"])

### Queries group by

In [36]:
import os
import sys
os.chdir("/home/foodmap/food101/")
sys.path.append(os.getcwd())
from processing.load_keyword_dicts import loadCategoryDict


cats = set(loadCategoryDict().values())
print len(cats)
for x in cats:
    print x
    results = es.search(index='stream', doc_type='tweet', size=100, body={
      'query': {
        'term': {
#             'img_flag': True,
            'img_categories.label': x,
#             'day': 20170629,
    #         'sort': [
    #         {'committed_date': {'order': 'desc'}}
    #       ],
         }
      }
    })
    print results["hits"]["total"]
#     if results["hits"]["total"]>0:
#         print results["hits"]["hits"]


101
grilled salmon
0
lobster roll sandwich
0
hamburger
4
[{u'_score': 3.8309722, u'_type': u'tweet', u'_id': u'880775644707160064', u'_source': {u'username': u'SwoutyyJae', u'lang': u'in', u'text_categories': [], u'city': u'kota kinabalu', u'text': u'#Dinner (@ Socialize in Kota Kinabalu) https://t.co/c8fUbMcVrS https://t.co/famGVPTLls', u'img_categories': [{u'score': 10.0, u'label': u'hamburger'}], u'hashtags': [u'#dinner'], u'img_flag': True, u'timestamp_ms': u'1498828252375', u'bounding_box': {u'type': u'Polygon', u'coordinates': [[[115.994996, 5.83293], [115.994996, 6.143625], [116.286089, 6.143625], [116.286089, 5.83293]]]}, u'coords': [116.11293971, 5.97639484], u'id_str': u'880775644707160064', u'country': u'malaysia', u'id': 880775644707160064, u'created_at_day': 20170630, u'created_at_datetime': u'2017-06-30T13:10:52', u'media_url': u'http://pbs.twimg.com/media/DDkkx5oWsAUbn-9.jpg'}, u'_index': u'stream'}, {u'_score': 3.754961, u'_type': u'tweet', u'_id': u'881920662092341248'

#### TEST image classifier

In [ ]:
# test some of the urls from the indexed tweets
# by reclassifying it with the IMG classifier hosted on our machine
request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
for tweet in results["hits"]["hits"]:
    img_url = tweet["_source"]["media_url"]
    res = requests.get(request_string + img_url)

    print tweet["_source"]["text"]
    print img_url
    print res.content
    print json.loads(res.text)["guessed"]
    print get_image_category(img_url)
    print 
    print

In [ ]:
# import requests
# import json

# response = requests.get('http://test.tripbuilder.isti.cnr.it:5000/sync')
# json_data = json.loads(response.text)
# json_data